# 3D Image Visualization with Dash

Installation: `pip install dash dash-vtk dash-bootstrap-components`

Documentation: [3D Visualization explained (Dash)](https://dash.plotly.com/vtk/intro)

In [ ]:
import dash
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
from dash import dcc
from dash import html
import dash_vtk
import base64
import tifffile
import io

app = dash.Dash(__name__, assets_url_path="/assets")

app.layout = dbc.Container(
    fluid=True,
    style={"height": "100vh"},
    children=[
        html.Div([
            dcc.Upload(
                id='upload-data',
                children=html.Div(['Click to Open or Drag and Drop Files']),
                multiple=False
            ),
            html.Div(id='output-data-upload'),
        ]),
        html.Div(
            dash_vtk.View(id="vtk-view"), 
            id="vtk-container",
        ),
    ],
)

def upload_file(contents):
    _, content_string = contents.split(',')
    buffer = base64.b64decode(content_string)
    image = tifffile.imread(io.BytesIO(buffer))
    return dash_vtk.VolumeDataRepresentation(
        dimensions=image.shape,
        scalars=image.ravel(order="F"),
    )

@app.callback(Output('vtk-view', 'children'), Input('upload-data', 'contents'))
def update_output(contents):
    if contents is not None:
        return upload_file(contents)
    
app.run(jupyter_mode="tab")  # The app will be running at http://localhost:8050